# 基于LLM bagging 的质控+ 可解释
> zhenkun.shi@tib.cas.cn   
> 2025-05-15 

## 1. 导入必要的包

In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../../')
from config import conf as cfg
import pandas as pd
import json
import plotly.graph_objects as go
from tools import btools
from tools import uniprottool as uptool
from tqdm import tqdm
import rxnrecer as production

from modules.llm import chat as llmchat
import tools.bioFunctionLib as bfl
from IPython.display import HTML
from pandarallel import pandarallel # 导入pandaralle
FIRST_TIME_RUN = False
pandarallel.initialize(progress_bar=False)
%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. 加载数据

In [2]:
ds_test = pd.read_feather(cfg.FILE_DS_TEST)
ds_test.head(3)

,uniprot_id,seq,reaction_id,ec_number,functionCounts,ec_specific_level,isenzyme,label
0,A9JLI2,MLGLQIFTLLSIPTLLYTYEIEPLERTSTPPEKEFGYWCTYANHCR...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A9JLI3,MRFFSYLGLLLAGLTSLQGFSTDNLLEEELRYWCQYVKNCRFCWTC...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,A9JLI5,MLVIFLGILGLLANQVLGLPTQAEGHLRSTDNPPQEELGYWCTYME...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 3. RXNRECer EXP

In [3]:
if FIRST_TIME_RUN:
    res_ecrecer = production.step_by_step_prediction(input_data=ds_test[['uniprot_id', 'seq']], Ensemble=True, batch_size=100)
    res_ecrecer.to_feather('res/res_ecrecer.feather')
    res_ecrecer = res_ecrecer.rename(columns={'input_id':'uniprot_id'})
    res_ecrecer = res_ecrecer.merge(ds_test[['uniprot_id', 'seq', 'reaction_id']], on='uniprot_id', how='left').rename(columns={'reaction_id':'rxn_groundtruth'})
else:
    res_ecrecer = pd.read_feather('res/res_ecrecer.feather')
    
res_ecrecer.head(3)

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
0,A9JLI2,-,"{'-': 0.999997, 'RHEA:10012': None, 'RHEA:1002...",MLGLQIFTLLSIPTLLYTYEIEPLERTSTPPEKEFGYWCTYANHCR...,-
1,A9JLI3,-,"{'-': 0.999998, 'RHEA:10012': None, 'RHEA:1002...",MRFFSYLGLLLAGLTSLQGFSTDNLLEEELRYWCQYVKNCRFCWTC...,-
2,A9JLI5,-,"{'-': 1.0, 'RHEA:10012': None, 'RHEA:10024': N...",MLVIFLGILGLLANQVLGLPTQAEGHLRSTDNPPQEELGYWCTYME...,-


In [4]:
res_ecrecer['RXNRECer_with_prob'] = res_ecrecer.RXNRECer_with_prob.apply(lambda x: {k: v for k, v in x.items() if v is not None})
res_ecrecer.head(3)

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
0,A9JLI2,-,{'-': 0.999997},MLGLQIFTLLSIPTLLYTYEIEPLERTSTPPEKEFGYWCTYANHCR...,-
1,A9JLI3,-,{'-': 0.999998},MRFFSYLGLLLAGLTSLQGFSTDNLLEEELRYWCQYVKNCRFCWTC...,-
2,A9JLI5,-,{'-': 1.0},MLVIFLGILGLLANQVLGLPTQAEGHLRSTDNPPQEELGYWCTYME...,-


In [7]:
# res_ecrecer.to_pickle('res/res_ecrecer.pkl')

In [5]:
filtered_data = {k: v for k, v in res_ecrecer.RXNRECer_with_prob.values[16].items() if v is not None}
filtered_data

{'-': 0.980291, 'RHEA:21744': 0.7777, 'RHEA:22104': 0.98089599609375}

In [9]:

# res_ecrecer.to_pickle(cfg.FILE_CASE_RESULTS_RXNRECER_REACTION)

## 4. Quality Control

In [6]:
## 4. Load RXN
rxn = pd.read_feather(cfg.FILE_RHEA_REACTION)
rxn.head(3)

,reaction_id,equation,chebi_id,ec_number,equation_chebi,equation_smiles
0,RHEA:22636,dCTP + H2O = dCMP + diphosphate + H(+),CHEBI:61481;CHEBI:15377;CHEBI:57566;CHEBI:3301...,EC:3.6.1.9;EC:3.6.1.12;EC:3.6.1.65,CHEBI:61481 + CHEBI:15377 = CHEBI:57566 + CHEB...,Nc1ccn([C@H]2C[C@H](O)[C@@H](COP([O-])(=O)OP([...
1,RHEA:22640,NADP(+) + sphinganine = 3-oxosphinganine + H(+...,CHEBI:58349;CHEBI:57817;CHEBI:58299;CHEBI:1537...,EC:1.1.1.102,CHEBI:58349 + CHEBI:57817 = CHEBI:58299 + CHEB...,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...
2,RHEA:22644,O2 + protopine + reduced [NADPH--hemoprotein r...,CHEBI:15379;CHEBI:16415;CHEBI:57618;CHEBI:1710...,EC:1.14.14.98,CHEBI:15379 + CHEBI:16415 + CHEBI:57618 = CHEB...,O=O.CN1CCc2cc3OCOc3cc2C(=O)Cc2ccc3OCOc3c2C1.Cc...


In [7]:
res_ecrecer.iloc[11:25]

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
11,P0DTH6,-,{'-': 0.999998},MKKIFLTMTPLPYMNHLPENHYFLSLTVVISIIHLFTTCVPQHNHS...,-
12,Q7XQ85,RHEA:21744;-,"{'-': 0.9853789806365967, 'RHEA:21744': 0.997593}",MAYQGIDLLSTKAAGDDHGENSSYFDGWKAYDTNPFDLRHNRGGVI...,RHEA:21744
13,A0A0P0WIY3,RHEA:21744;-,"{'-': 0.7777, 'RHEA:21744': 0.990111}",MVGRMLSSPEPTLSTMAMSAAHGEDSPYFAGWRAYDEDPYDPITNP...,RHEA:21744
14,Q5W6F9,RHEA:21744;-,"{'-': 0.7777, 'RHEA:21744': 0.989836}",MGVKLLADGCAGASSSPALSRVATSAAHGEGSPYFAGWKAYDEDPY...,RHEA:21744
15,A0A0P0UZP7,RHEA:21744;-,"{'-': 0.7777, 'RHEA:21744': 0.97918701171875}",MIMSGFHIGIYTSICLYIPLPHLEPWIISSHTPKNLNLLDCLLYCS...,RHEA:21744
16,Q9SNN8,RHEA:22104;-;RHEA:21744,"{'-': 0.980291, 'RHEA:21744': 0.7777, 'RHEA:22...",MRRSGNGGAAKKKKKRSASAASERRPRADGGMRIVVPLQGVVQGRG...,RHEA:21744
17,L0TBY6,-;RHEA:20049,"{'-': 0.99999, 'RHEA:20049': 0.9893050193786621}",MTHREELLPPMKWDAWGDPAAAKPLSDGVRSLLKQVVGLADSEQPE...,-
18,M5AWY0,RHEA:34415;-;RHEA:56596,"{'-': 0.7777, 'RHEA:34415': 0.9994519948959351...",MQAGFFHTPYNLPTRTARQMFDWSLKLAQVCDEAGFADFMIGEHST...,RHEA:34415
19,Q6I621,-,{'-': 0.999998},MWKGFLSKLPRKTSASGRGADLDSGQCSNGAGNGNPIQRTSSCGSI...,-
20,Q88FI7,RHEA:18049;RHEA:23352;RHEA:10212;RHEA:12268,"{'RHEA:10212': 0.7777, 'RHEA:12268': 0.7777, '...",MNQESISQSIAIVHPITLSHGRNAEVWDTDGKRYIDFVGGIGVLNL...,RHEA:12601;RHEA:10212


In [ ]:
def get_rxn_detail(rxn_id, rxn_bank):
    if rxn_id == '-':
        return {
            'reaction id': '-',
            'reaction equation': '-'
        }
    
    rxn_record = 
    [rxn_bank.reaction_id == rxn_id]
    
    if rxn_record.empty:
        return {}  # Avoid errors if the record is missing

    rxn_record = rxn_record.iloc[0]  # Take the first match

    return {
        'reaction id': rxn_record.reaction_id,
        'reaction equation': rxn_record.equation,
        'reaction equation in ChEBI format': rxn_record.equation_chebi,
        'reaction equation in SMILES format': rxn_record.equation_smiles,
        'reaction associated Enzyme Commission Number': rxn_record.ec_number
    }

def make_make_query(uniprot_id, seq, rxn_id, rxn_bank, prompt):
    if not rxn_id or cfg.SPLITER not in rxn_id:
        return {}

    rxns = rxn_id.split(cfg.SPLITER)



    query_info = {
        'protein information': {
            'uniprot id': uniprot_id,
            'protein amino acid sequence': seq
        },
        'reaction information': {
            f'candidate reaction {i+1}': get_rxn_detail(rxn, rxn_bank)
            for i, rxn in enumerate(rxns)
        }
    }

    llm_query = prompt  + json.dumps(query_info, indent=4, ensure_ascii=False)

    return llm_query


In [10]:
prompt0 = """You are a biochemical expert. Your task is to analyze a protein and determine which reaction(s) it is most likely to catalyze based on the provided data.

### Guidelines:
1. **Input Data:**
- **Protein Information:** Includes the Uniprot ID and the protein's amino acid sequence.
- **Reaction Information:** A list of candidate reactions. Each reaction is provided in:
    - Standard reaction equation
    - Reaction equation in ChEBI format
    - Reaction equation in SMILES format

2. **Sequence Analysis & Functional Inference:**
- **Basic Properties:** Consider the protein's length, molecular weight, and amino acid composition to see if they match known enzyme classes.
- **Domain & Active Site Identification:** Look for conserved domains and key catalytic residues (or cofactor-binding motifs) that are critical for catalysis.
- **Structural Considerations:** Evaluate whether the predicted secondary/tertiary structure supports the formation of an active site suitable for the reaction’s substrate.
- **Database & Homology Evidence:** Use similarities to annotated proteins and known enzyme families to infer likely catalytic activity.

3. **Handling Non-Catalytic Cases:**
- A candidate reaction denoted by a single dash ("-") indicates that the protein is predicted not to catalyze that particular reaction.
- **Important:** If all candidate reactions are marked with a dash, or if your analysis suggests that none of the reactions with valid IDs are likely catalyzed by the protein, then include the dash candidate reaction in the output to indicate that the protein is not catalytically active for any provided reaction.

4. **Multiple Reactions:**
- Proteins may have multiple active sites. If applicable, select more than one reaction.

5. **Ranking:**
- Rank the selected reactions by likelihood (1 being the most likely).

6. **Justification:**
- Provide a brief explanation for each selection, citing relevant sequence features (e.g., conserved domains, active site residues, structural motifs) and reaction details.

### Expected Output (in JSON format):
```json
[
    {
        "reaction_id": "xxx",
        "selected": "yes" or "no",
        "rank": <integer>,         // 1 for the most likely, 2 for the next, etc.
        "confidence": <number>,      // Value between 0 and 1
        "reason": "Explanation based on sequence and reaction data."
    },
    ...
]
"""

prompt1 = """
You are a biochemical expert. Given a protein sequence and a set of potential catalyzed reactions, please analyze and determine the most likely catalyzed reactions. 
    The information is provided in JSON format.
    Important note: If a reaction is represented by a single dash ("-"), it means the protein cannot catalyze a reaction. 
    Please consider this in your analysis.
    
Guidelines:
    1. **Handling Non-Catalytic Cases:**
    - A candidate reaction denoted by a single dash ("-") indicates that the protein is predicted not to catalyze that particular reaction.
    - **Important:** If all candidate reactions are marked with a dash, or if your analysis suggests that none of the reactions with valid IDs are likely catalyzed by the protein, then include the dash candidate reaction in the output to indicate that the protein is not catalytically active for any provided reaction.

    2. **Multiple Reactions:**
    - Proteins may have multiple active sites. If applicable, select more than one reaction.

    3. **Ranking:**
    - Rank the selected reactions by likelihood (1 being the most likely).

    4. **Justification:**
    - Provide a brief explanation for each selection, citing relevant sequence features (e.g., conserved domains, active site residues, structural motifs) and reaction details.
    
    5. **Confidence Scoring:**  
    - For each reaction, assign a confidence score between 0 and 1 based on how well the protein sequence matches known features (e.g., conserved domains, motifs, known homologs) for the reaction. Use this score to determine the ranking.  
    - If none of the reactions are plausible, assign the highest confidence to the dash ("-") reaction to indicate a strong negative prediction.

    
Expected Output (in JSON format):
    ```json
    [
        {
            "reaction_id": "xxx",
            "selected": "yes" or "no",
            "rank": <integer>,         // 1 for the most likely, 2 for the next, etc.
            "confidence": <number>,      // Value between 0 and 1
            "reason": "Explanation based on sequence and reaction data."
        },
        ...
    ]
⚠️ Please do not use any other field name such as `"analysis"`. Only `"results"` is allowed.

Input:
"""

prompt2 = """
You are a biochemical expert. Your task is to analyze a protein and determine which reaction(s) it is most likely to catalyze based on the provided data in JSON format.

Important Notes:
1. If a reaction is represented by a single dash ("-"), it means the protein is predicted not to catalyze that reaction. However, if all candidate reactions are marked with a dash, or if your analysis suggests that none of the reactions with valid IDs are likely catalyzed by the protein, then include the dash candidate reaction in your output to indicate that the protein is not catalytically active for any provided reaction.
2. Proteins may have multiple active sites. If applicable, select more than one reaction.
3. For each reaction, provide a brief explanation citing relevant sequence features (e.g., conserved domains, active site residues, structural motifs) and reaction details.

Output your final answer in JSON format using the following template:
{
    "reaction_id": "xxx",
    "selected": "yes" or "no",
    "rank": <integer>,         // 1 for the most likely, 2 for the next, etc.
    "confidence": <number>,      // A value between 0 and 1
    "reason": "Explanation based on sequence and reaction data."
}

Input:
"""


prompt3 = """
You are a biochemical expert. Your task is to analyze a protein and determine which reaction(s) it is most likely to catalyze based on the provided data in JSON format.

Guidline:
1. **Structural Chemistry Analysis**
   - Parse SMILES to identify reaction centers (e.g., bond cleavage sites)
   $$ \text{Example: } \text{C-S bond in } \mathrm{CHEBI:59789} \rightarrow \mathrm{CH_3S^- + adenosyl} $$
   - Map ChEBI IDs to functional groups (e.g., thioether in CHEBI:59789)

2. **Sequence-Structure Mapping**
   - Detect catalytic triads using PROSITE patterns:
     $$ \text{Cys-X-X-Cys} \Rightarrow \text{Sulfur metabolism} $$
     $$ \text{Ser-Lys-Asp} \Rightarrow \text{Hydrolase activity} $$
   - Verify cofactor binding with PROSITE PDOC00005

3. **Dynamic Compatibility Check**
   - Calculate active site volume using:
     $$ V_{\text{site}} = \frac{4}{3}\pi r_{\text{substrate}}^3 \pm 15\% $$
   - Validate stereochemical constraints from SMILES chirality tags (@, @@)

4. **Conflict Resolution Protocol**
   - Prioritize reaction chemistry over EC numbers when conflicts exist
   - Use UniProtKB/Swiss-Prot annotations (e.g., FT ACT_SITE) as tiebreaker

**Important Notes:**
- If a reaction is represented by a single dash ("-"), it means the protein is predicted not to catalyze that reaction. However, if all candidate reactions are marked with a dash, or if your analysis suggests that none of the reactions with valid IDs are likely catalyzed by the protein, then include the dash candidate reaction in your output to indicate that the protein is not catalytically active for any provided reaction.
- Proteins may have multiple active sites. If applicable, select more than one reaction.
- For each reaction, provide a brief explanation citing relevant sequence features (e.g., conserved domains, active site residues, structural motifs) and reaction details.

Output your final answer in JSON format using the following template:
{
    "reaction_id": "xxx",
    "selected": "yes" or "no",
    "rank": <integer>,         // 1 for the most likely, 2 for the next, etc.
    "confidence": <number>,      // A value between 0 and 1
    "reason": "Explanation based on sequence and reaction data."
}

Input:

"""


prompt3 = """
You are a biochemical expert. Your task is to analyze a protein and determine which reaction(s) it is most likely to catalyze based on the provided data in JSON format.

Guidline:
1. **Structural Chemistry Analysis**
   - Parse SMILES to identify reaction centers (e.g., bond cleavage sites)
   $$ \text{Example: } \text{C-S bond in } \mathrm{CHEBI:59789} \rightarrow \mathrm{CH_3S^- + adenosyl} $$
   - Map ChEBI IDs to functional groups (e.g., thioether in CHEBI:59789)

2. **Sequence-Structure Mapping**
   - Detect catalytic triads using PROSITE patterns:
     $$ \text{Cys-X-X-Cys} \Rightarrow \text{Sulfur metabolism} $$
     $$ \text{Ser-Lys-Asp} \Rightarrow \text{Hydrolase activity} $$
   - Verify cofactor binding with PROSITE PDOC00005

3. **Dynamic Compatibility Check**
   - Calculate active site volume using:
     $$ V_{\text{site}} = \frac{4}{3}\pi r_{\text{substrate}}^3 \pm 15\% $$
   - Validate stereochemical constraints from SMILES chirality tags (@, @@)

4. **Conflict Resolution Protocol**
   - Prioritize reaction chemistry over EC numbers when conflicts exist
   - Use UniProtKB/Swiss-Prot annotations (e.g., FT ACT_SITE) as tiebreaker

**Important Notes:**
- If a reaction is represented by a single dash ("-"), it means the protein is predicted not to catalyze that reaction. However, if all candidate reactions are marked with a dash, or if your analysis suggests that none of the reactions with valid IDs are likely catalyzed by the protein, then include the dash candidate reaction in your output to indicate that the protein is not catalytically active for any provided reaction.
- Proteins may have multiple active sites. If applicable, select more than one reaction.
- For each reaction, provide a brief explanation citing relevant sequence features (e.g., conserved domains, active site residues, structural motifs) and reaction details.

Output your final answer in JSON format using the following template:
{
    "reaction_id": "xxx",
    "selected": "yes" or "no",
    "rank": <integer>,         // 1 for the most likely, 2 for the next, etc.
    "confidence": <number>,      // A value between 0 and 1
    "reason": "Explanation based on sequence and reaction data."
}

Input:

"""

prompt4 = """
You are a biochemical expert. Your task is to analyze a protein and determine which reaction(s) it is most likely to catalyze based on the provided data in JSON format.

Guidline:
1. **Omni-enzyme Structural Profiling**
   - Perform topological analysis of SMILES using:
     $$ \text{Graph neural networks for reaction center detection} $$
     $$ \text{Electrostatic potential mapping (EPM) for charge complementarity} $$
   - Cross-reference ChEBI IDs with MetaCyc reaction pathways:
     $$ \forall R \in \text{Reactions}, \exists \text{Substrate-Enzyme PCMatrix} \geq 0.7 $$

2. **Quantum Biochemical Evaluation**
   - Calculate catalytic proficiency using:
     $$ k_{\text{cat}}/K_m = \frac{k_B T}{h} e^{-\Delta G^\ddagger/RT} $$
   - Simulate transition states with:
     $$ \text{QM/MM modeling of active site frontier orbitals} $$
     $$ \text{Reaction coordinate analysis (IRC)} $$

3. **Evolutionary Dynamics Integration**
   - Calculate phylogenetic catalytic propensity:
     $$ P_{\text{cat}} = \frac{\sum \text{Consurf scores}}{\text{Active site residues}} \times \text{EC_specific_phyletic_pattern} $$
   - Detect convergent evolution signatures through:
     $$ \text{Parallel substitution analysis in catalytic residues} $$
     $$ \text{Structural phylogeny using DALI Z-scores} $$

4. **Multi-scale Mechanistic Validation**
   - Verify allosteric regulation potential:
     $$ \text{Ensemble docking across conformational states} $$
     $$ \text{Principal component analysis of MD trajectories} $$
   - Cross-validate with omics data:
     $$ \text{Co-expression with metabolic pathway genes (r ≥ 0.85)} $$
     $$ \text{Proteomic colocalization evidence} $$

**Important Notes:**
- If a reaction is represented by a single dash ("-"), it means the protein is predicted not to catalyze that reaction. However, if all candidate reactions are marked with a dash, or if your analysis suggests that none of the reactions with valid IDs are likely catalyzed by the protein, then include the dash candidate reaction in your output to indicate that the protein is not catalytically active for any provided reaction.
- Proteins may have multiple active sites. If applicable, select more than one reaction.
- For each reaction, provide a brief explanation citing relevant sequence features (e.g., conserved domains, active site residues, structural motifs) and reaction details.

Output your final answer in JSON format using the following template:
{
    "reaction_id": "xxx",
    "selected": "yes" or "no",
    "rank": <integer>,         // 1 for the most likely, 2 for the next, etc.
    "confidence": <number>,    // A value between 0 and 1
    "reason": "Explanation based on sequence and reaction data."
}

Input:
"""


prompt5 = """
You are a biochemical expert. Your task is to analyze a protein and determine which reaction(s) it is most likely to catalyze based on the provided data in JSON format.

Guidline:
1. **Literature Evidence Triangulation** ★最高优先级
   a. UniProt/Sequence-based search:
     $$ \text{IF } \exists \text{ PMIDs } (P|E) \rightarrow \text{ReactID} \text{ THEN confidence} += 0.4 $$
   b. Reaction-centric search:
     $$ \text{Match(Equation|ChEBI|SMILES) → PMIDs → Enzyme } \cap \text{Input} $$
   c. EC number validation:
     $$ \text{Validate EC}_{obs} = \frac{\text{PMID}_{\text{EC-match}}}{\text{PMID}_{\text{EC-total}}} \geq 0.6 $$

2. **Structural Catalytic Competence**
   - SMILES拓扑指纹匹配:
     $$ \text{Tanimoto} \geq 0.85 \text{ with Catalophore model} $$
   - 电荷互补性验证:
     $$ \Delta E_{\text{elec}} = \sum \frac{q_i q_j}{4\pi\epsilon r_{ij}} \leq -15 \text{ kJ/mol} $$

3. **Quantum Transition State Analysis**
   - 催化效率计算:
     $$ \ln(k_{\text{cat}}/K_m) \propto -\Delta G^\ddagger_{\text{calc}} + \text{文献修正项} $$
   - 轨道相互作用验证:
     $$ \text{HOMO-LUMO gap匹配度} \geq 82\% $$

4. **Evolutionary & Omics Validation**
   - 系统发育可信度:
     $$ P_{\text{cat}} = \text{Consurf} \times \text{EC}_{phylo} \geq 7.5 $$
   - 组学相关性:
     $$ r_{\text{coex}} \geq 0.9 \text{ AND } \text{LocalScore} \geq 0.8 $$

**Important Notes:**
- If a reaction is represented by a single dash ("-"), it means the protein is predicted not to catalyze that reaction. However, if all candidate reactions are marked with a dash, or if your analysis suggests that none of the reactions with valid IDs are likely catalyzed by the protein, then include the dash candidate reaction in your output to indicate that the protein is not catalytically active for any provided reaction.
- Proteins may have multiple active sites. If applicable, select more than one reaction.
- For each reaction, provide a brief explanation citing relevant sequence features (e.g., conserved domains, active site residues, structural motifs) and reaction details.

Output your final answer in JSON format using the following template:
{
    "reaction_id": "xxx",
    "selected": "yes" or "no",
    "rank": <integer>,         // 1 for the most likely, 2 for the next, etc.
    "confidence": <number>,    // A value between 0 and 1
    "reason": "Explanation based on sequence and reaction data."
}

Input:

"""

In [11]:
schema = {
    "type": "function",   
    "name": "return_predictions",
    "parameters": {
        "type": "object",
        "properties": {
            "results": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "reaction_id":  {"type": "string"},
                        "selected":     {"type": "string"},
                        "rank":         {"type": ["integer", "null"]},
                        "confidence":   {"type": "number"},
                        "reason":       {"type": "string"}
                    },
                    "required": ["reaction_id", "selected", "confidence", "reason"]
                }
            }
        },
        "required": ["results"]
    }
}


system_prompt = """
You are a biochemical expert. Given a protein sequence and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein.
You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.
Key considerations:
1. **Non-Catalytic Cases:**
   - A reaction represented by a single dash ("-") indicates a predicted lack of catalytic activity.
   - If all reactions are marked as "-", or none of the valid reactions are plausible, select the dash ("-") reaction to indicate non-catalytic function.

2. **Multiple Reactions:**
   - Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.

3. **Ranking:**
   - Assign a likelihood-based ranking to selected reactions (1 = most likely).

4. **Justification:**
   - Provide a brief explanation for each decision, referencing conserved domains, active site residues, structural motifs, or known catalytic mechanisms.

5. **Confidence Score:**
   - Assign a confidence score between 0 and 1 based on how well the protein sequence aligns with known catalytic features for each reaction.
   - If no valid reaction is likely, assign the highest confidence to the dash ("-") reaction.

Output format (strictly JSON):
[
    {
        "reaction_id": "xxx",
        "selected": "yes" or "no",
        "rank": <integer>,     // Only for selected = "yes"
        "confidence": <float>, // Between 0 and 1
        "reason": "Explanation based on sequence and reaction data."
    },
    ...
]

⚠️ Only use the key 'results' in the output. Do not include any extra explanation or metadata.
"""

In [12]:
system_prompt = """
You are a biochemical expert. Given a protein sequence and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein.
You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.
Key considerations:
1. **Non-Catalytic Cases:**
   - A reaction represented by a single dash ("-") indicates a predicted lack of catalytic activity.
   - If all reactions are marked as "-", or none of the valid reactions are plausible, select the dash ("-") reaction to indicate non-catalytic function.

2. **Multiple Reactions:**
   - Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.

3. **Ranking:**
   - Assign a likelihood-based ranking to selected reactions (1 = most likely).

4. **Justification:**
   - Provide a brief explanation for each decision, referencing conserved domains, active site residues, structural motifs, or known catalytic mechanisms.

5. **Confidence Score:**
   - Assign a confidence score between 0 and 1 based on how well the protein sequence aligns with known catalytic features for each reaction.
   - If no valid reaction is likely, assign the highest confidence to the dash ("-") reaction.

Output format (strictly JSON):
[
    {
        "reaction_id": "xxx",
        "selected": "yes" or "no",
        "rank": <integer>,     // Only for selected = "yes"
        "confidence": <float>, // Between 0 and 1
        "reason": "Explanation based on sequence and reaction data."
    },
    ...
]

⚠️ Only use the key 'results' in the output. Do not include any extra explanation or metadata.
"""



'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'

In [7]:
! source ~/.zshrc

In [ ]:

APIKEY = os.environ.get("OPENROUTER_API_KEY")
cli5 = llmchat.Chat(name='openai/gpt-4.1', 
               url='https://openrouter.ai/api/v1', 
               api_key = APIKEY,
            #    proxy='http://172.16.10.2:7897'
)


def make_make_user_query(uniprot_id, seq, rxn_id, rxn_bank):
    if not rxn_id or cfg.SPLITER not in rxn_id:
        return {}
    rxns = rxn_id.split(cfg.SPLITER)

    query_info = {
        'protein information': {
            'uniprot id': uniprot_id,
            'protein amino acid sequence': seq
        },
        'reaction information': {
            f'candidate reaction {i+1}': get_rxn_detail(rxn, rxn_bank)
            for i, rxn in enumerate(rxns)
        }
    }

    llm_query =  json.dumps(query_info, indent=4, ensure_ascii=False)

    return llm_query

def session_chat(chat_bot, ix, system_prompt, debug=True):
    
    query_user = make_make_user_query(uniprot_id=res_ecrecer.iloc[ix].uniprot_id, seq=res_ecrecer.iloc[ix].seq, rxn_id=res_ecrecer.iloc[ix].RXNRECer, rxn_bank=rxn)
    if debug:
        print(system_prompt)
        print(query_user)
   
    response = chat_bot.chat(message=query_user, system_prompt=system_prompt, response_format={"type": "json_object"})   # 关键行，指定返回格式为json_object)
    print(response)
    records = json.loads(response.choices[0].message.content)
    
    json_results = records
    df_results = pd.DataFrame(records['results']).fillna(float('inf'))
    
    
    if debug:

         print(response)
         print(json_results)
    
    return df_results, json_results



In [17]:
res_df, res_json = session_chat(chat_bot=cli5, ix=12, system_prompt=system_prompt)


You are a biochemical expert. Given a protein sequence and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein.
You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.
Key considerations:
1. **Non-Catalytic Cases:**
   - A reaction represented by a single dash ("-") indicates a predicted lack of catalytic activity.
   - If all reactions are marked as "-", or none of the valid reactions are plausible, select the dash ("-") reaction to indicate non-catalytic function.

2. **Multiple Reactions:**
   - Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.

3. **Ranking:**
   - Assign a likelihood-based ranking to selected reactions (1 = most likely).

4. **Justification:**
   - Provide a brief explanation for each decision, referencing conserved domains, active site residues, structural mo

In [18]:
res_json

{'results': [{'reaction_id': 'RHEA:21744',
   'selected': 'yes',
   'rank': 1,
   'confidence': 0.98,
   'reason': "The UniProt ID Q7XQ85 corresponds to the protein 1-aminocyclopropane-1-carboxylate synthase (ACC synthase), which catalyzes the conversion of S-adenosyl-L-methionine to 1-aminocyclopropane-1-carboxylate, H(+), and S-methyl-5'-thioadenosine (EC 4.4.1.14). The sequence contains conserved motifs and active site residues characteristic of the ACC synthase family, including the pyridoxal phosphate (PLP)-binding domain, which is essential for this lyase activity. This matches the reaction RHEA:21744 with high confidence."},
  {'reaction_id': '-',
   'selected': 'no',
   'confidence': 0.01,
   'reason': 'The protein sequence and UniProt annotation strongly support catalytic activity as ACC synthase. There is no evidence for a non-catalytic function.'}]}

In [33]:
print(system_prompt)


You are a biochemical expert. Given a protein sequence and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein.
You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.
Key considerations:
1. **Non-Catalytic Cases:**
   - A reaction represented by a single dash ("-") indicates a predicted lack of catalytic activity.
   - If all reactions are marked as "-", or none of the valid reactions are plausible, select the dash ("-") reaction to indicate non-catalytic function.

2. **Multiple Reactions:**
   - Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.

3. **Ranking:**
   - Assign a likelihood-based ranking to selected reactions (1 = most likely).

4. **Justification:**
   - Provide a brief explanation for each decision, referencing conserved domains, active site residues, structural mo

In [42]:
res_json

{'results': [{'reaction_id': 'RHEA:21744',
   'selected': 'yes',
   'rank': 1,
   'confidence': 0.98,
   'reason': "The UniProt ID Q7XQ85 corresponds to the protein 1-aminocyclopropane-1-carboxylate synthase (ACC synthase), which catalyzes the conversion of S-adenosyl-L-methionine to 1-aminocyclopropane-1-carboxylate, H(+), and S-methyl-5'-thioadenosine (EC 4.4.1.14). The sequence contains conserved motifs and active site residues characteristic of the ACC synthase family, including the pyridoxal phosphate (PLP)-binding domain, which is essential for this lyase activity. This matches the reaction RHEA:21744 with high confidence."},
  {'reaction_id': '-',
   'selected': 'no',
   'confidence': 0.01,
   'reason': 'The protein sequence and UniProt annotation strongly support catalytic activity as ACC synthase. There is no evidence for a non-catalytic function.'}]}

In [ ]:
res_df, res_json = session_chat(chat_bot=cli5, ix=12, system_prompt=system_prompt)

In [272]:
session_chat(chat_bot=cli5, ix=17)

,reaction_id,selected,confidence,reason,rank
0,RHEA:20049,no,0.15,The protein L0TBY6 (UniProt) is annotated as a...,inf
1,-,yes,0.95,The sequence lacks identifiable catalytic doma...,1.0


In [273]:
session_chat(chat_bot=cli5, ix=29)

,reaction_id,selected,rank,confidence,reason
0,RHEA:52584,yes,1.0,0.98,"The UniProt entry A7B3K3 corresponds to BaiA1,..."
1,RHEA:52596,yes,2.0,0.97,This is another substrate-specific reaction fo...
2,RHEA:52588,yes,3.0,0.96,This reaction is also catalyzed by 3α-hydroxys...
3,RHEA:52592,yes,4.0,0.96,Deoxycholoyl-CoA is a known substrate for BaiA...
4,RHEA:55380,yes,5.0,0.99,This is the generic reaction for 3α-hydroxyste...
5,RHEA:11728,no,inf,0.10,Cyclopentanol dehydrogenase (EC 1.1.1.163) is ...
6,RHEA:10044,no,inf,0.10,Cyclohexanol dehydrogenase (EC 1.1.1.245) is n...
7,RHEA:17397,no,inf,0.10,3-oxoacyl-[ACP] reductase (EC 1.1.1.100) is a ...
8,RHEA:47496,no,inf,0.20,This reaction is catalyzed by 3α-hydroxysteroi...
9,-,no,inf,0.01,"The protein is clearly catalytic, with strong ..."


In [274]:
session_chat(chat_bot=cli5, ix=94)

,reaction_id,selected,confidence,reason,rank
0,RHEA:55056,no,0.15,"This reaction is catalyzed by EC 1.14.14.49, a...",inf
1,-,yes,0.95,"The protein sequence lacks conserved domains, ...",1.0
2,RHEA:17149,no,0.10,This is a generic cytochrome P450 monooxygenas...,inf
3,RHEA:43728,no,0.10,"This reaction is catalyzed by EC 1.14.14.127, ...",inf


In [51]:
res_df_17, res_json_17 = session_chat(chat_bot=cli5, ix=17, system_prompt=system_prompt)

{
    "protein information": {
        "uniprot id": "L0TBY6",
        "protein amino acid sequence": "MTHREELLPPMKWDAWGDPAAAKPLSDGVRSLLKQVVGLADSEQPELDPAQVQLRPSALSGADHDALARIVGTEYFRTADRDRLLHAGGKSTPDLLRRKDTGVQDAPDAVLLPGGPNGGGRRRRHLALLLRPRHCRGPVWWRHQRRWWA"
    },
    "reaction information": {
        "candidate reaction 1": {
            "reaction id": "-",
            "reaction equation": "-"
        },
        "candidate reaction 2": {
            "reaction id": "RHEA:20049",
            "reaction equation": "1-(5-phospho-beta-D-ribosyl)-5'-AMP + H2O = 1-(5-phospho-beta-D-ribosyl)-5-[(5-phospho-beta-D-ribosylamino)methylideneamino]imidazole-4-carboxamide",
            "reaction equation in ChEBI format": "CHEBI:59457 + CHEBI:15377 = CHEBI:58435",
            "reaction equation in SMILES format": "O[C@H]1[C@@H](O)[C@@H](O[C@@H]1COP([O-])([O-])=O)n1cnc2c1ncn([C@@H]1O[C@H](COP([O-])([O-])=O)[C@@H](O)[C@H]1O)c2=N.[H]O[H]>>NC(=O)c1ncn([C@@H]2O[C@H](COP([O-])([O-])=O)[C@@H](O)[C@H]2O)c1\\\\N=

In [53]:
res_json_17

{'results': [{'reaction_id': '-',
   'selected': 'yes',
   'rank': 1,
   'confidence': 0.95,
   'reason': 'The protein sequence (UniProt L0TBY6) does not show significant homology to known enzymes catalyzing the candidate reaction (EC 3.5.4.19, phosphoribosylformylglycinamidine cyclo-ligase). No conserved domains, active site motifs, or sequence features characteristic of this enzyme class are present. Therefore, the most likely scenario is a lack of catalytic activity.'},
  {'reaction_id': 'RHEA:20049',
   'selected': 'no',
   'confidence': 0.1,
   'reason': 'No evidence of conserved domains or sequence motifs associated with EC 3.5.4.19 (phosphoribosylformylglycinamidine cyclo-ligase) is found in the provided sequence. The protein does not align with known catalytic features for this reaction.'}]}

In [56]:
print(system_prompt)


You are a biochemical expert. Given a protein sequence and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein.
You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.
Key considerations:
1. **Non-Catalytic Cases:**
   - A reaction represented by a single dash ("-") indicates a predicted lack of catalytic activity.
   - If all reactions are marked as "-", or none of the valid reactions are plausible, select the dash ("-") reaction to indicate non-catalytic function.

2. **Multiple Reactions:**
   - Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.

3. **Ranking:**
   - Assign a likelihood-based ranking to selected reactions (1 = most likely).

4. **Justification:**
   - Provide a brief explanation for each decision, referencing conserved domains, active site residues, structural mo

In [57]:
res_df_20, res_json_20 = session_chat(chat_bot=cli5, ix=20, system_prompt=system_prompt)

{
    "protein information": {
        "uniprot id": "Q88FI7",
        "protein amino acid sequence": "MNQESISQSIAIVHPITLSHGRNAEVWDTDGKRYIDFVGGIGVLNLGHCNPAVVEAIQAQATRLTHYAFNAAPHGPYLALMEQLSQFVPVSYPLAGMLTNSGAEAAENALKVARGATGKRAIIAFDGGFHGRTLATLNLNGKVAPYKQRVGELPGPVYHLPYPSADTGVTCEQALKAMDRLFSVELAVEDVAAFIFEPVQGEGGFLALDPAFAQALRRFCDERGILIIIDEIQSGFGRTGQRFAFPRLGIEPDLLLLAKSIAGGMPLGAVVGRKELMAALPKGGLGGTYSGNPISCAAALASLAQMTDENLATWGERQEQAIVSRYERWKASGLSPYIGRLTGVGAMRGIEFANADGSPAPAQLAKVMEAARARGLLLMPSGKARHIIRLLAPLTIEAEVLEEGLDILEQCLAELN"
    },
    "reaction information": {
        "candidate reaction 1": {
            "reaction id": "RHEA:18049",
            "reaction equation": "2-oxoglutarate + N(2)-acetyl-L-ornithine = L-glutamate + N-acetyl-L-glutamate 5-semialdehyde",
            "reaction equation in ChEBI format": "CHEBI:16810 + CHEBI:57805 = CHEBI:29985 + CHEBI:29123",
            "reaction equation in SMILES format": "[O-]C(=O)CCC(=O)C([O-])=O.CC(=O)N[C@@H](CCC[NH3+])C([O-])=O>>[NH3+][C@@H](CCC([O-

In [58]:
res_df_20

,reaction_id,selected,rank,confidence,reason
0,RHEA:18049,yes,1.0,0.98,The UniProt ID Q88FI7 corresponds to N-acetylo...
1,RHEA:23352,no,inf,0.15,This reaction is catalyzed by 4-aminobutyrate ...
2,RHEA:10212,no,inf,0.10,This reaction is catalyzed by 5-aminovalerate ...
3,RHEA:12268,no,inf,0.10,This reaction is catalyzed by putrescine trans...


In [ ]:
RHEA:12601;RHEA:10212

In [ ]:
res_ecrecer.iloc[30:55]

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
30,C0HK72,-,{'-': 1.0},RGICLEPKVVGPCKARIRRFYYDSETGKCTPFIYGGCGGNGNNFET...,-
31,A7AZH2,RHEA:55380;RHEA:52588;RHEA:52584;RHEA:52592;RH...,"{'-': 0.7777, 'RHEA:17397': 0.7777, 'RHEA:1883...",MNFGGFIMGRFDEKIMLVTGATSGIGRAVAIRAAKEGATVVAVGRN...,RHEA:47520
32,P0DX23,-;RHEA:14709;RHEA:32187;RHEA:54796,"{'-': 0.998382, 'RHEA:14709': 0.7777, 'RHEA:32...",MKVLVTGATSGLGRNAVEYLRNKGISVRATGRNEAMGKLLSKMGAE...,RHEA:14981;RHEA:14929;RHEA:24616;RHEA:24612
33,P0DX24,RHEA:35459;RHEA:43936;RHEA:43932;RHEA:43928;RH...,"{'-': 0.7777, 'RHEA:14709': 0.9953950047492981...",MGDPTLRTDLGRVLVTGGSGFVGANLVTTLLERGHEVRSFDRVPSP...,RHEA:14929;RHEA:14981
34,A0A7H0DNE2,RHEA:14709;RHEA:24076;RHEA:32187,"{'RHEA:14709': 0.9999309778213501, 'RHEA:24076...",MAVYAVTGGAGFLGRYIVKLLISADDVQEIRVIDIVEDPQPITSKV...,RHEA:24076;RHEA:14709
35,A0A7H0DNE2,RHEA:14709;RHEA:24076;RHEA:43932;RHEA:32187,"{'RHEA:14709': 0.9999309778213501, 'RHEA:24076...",MAVYAVTGGAGFLGRYIVKLLISADDVQEIRVIDIVEDPQPITSKV...,RHEA:24076;RHEA:14709
36,C0HK73,-,{'-': 0.999998},GSICLEPKVVGPCKAGIRRFYFDSETGKCTLFLYGGCKGNGNNFET...,-
37,F2XF94,RHEA:32539;-,"{'-': 0.7777, 'RHEA:32539': 0.9999420046806335}",MSVISIVPLVSKPCLYKSFISSTHEPKALRRPISTVGLCRRAKSVT...,RHEA:32539
38,R9QMW4,RHEA:25488;RHEA:25496;RHEA:32539;-,"{'-': 0.7777, 'RHEA:25488': 0.9992690086364746...",MSLISAVPLASSCVSKSLISSVREHKALRRAIATLQMSRPGKSVAA...,RHEA:32539;RHEA:25500
39,R9QMW8,RHEA:25488;RHEA:25496;RHEA:32539;-,"{'-': 0.7777, 'RHEA:25488': 0.9990990161895752...",MSLISAVPLASSCVSKSLISSVREHTALRRAIATLQMSRRGKSVAA...,RHEA:32539;RHEA:25500


In [62]:
res_ecrecer[res_ecrecer.rxn_groundtruth!='-'].iloc[30:55]

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
89,U5NH37,RHEA:39895;-,"{'-': 0.7777, 'RHEA:39895': 0.9999889731407166}",MGSQETNLPPHVLIFPLPIQGHVNSMLRLAELLCLAELDITFIVSE...,RHEA:39895
91,F1P963,RHEA:31583;RHEA:67392;RHEA:67604;RHEA:65396;RH...,"{'-': 0.7777, 'RHEA:31575': 0.9983620047569275...",MGTSRLYTLVLVLQPERVLLGMKKRGFGAGRWNGFGGKVQEGETIE...,RHEA:31583;RHEA:31575;RHEA:67604;RHEA:67608;RH...
92,Q7ZWC3,RHEA:31583;RHEA:67392;RHEA:67604;RHEA:65396;RH...,"{'-': 0.7777, 'RHEA:31575': 0.9945039749145508...",MFTSKLLTLVLVVQPGRVLLGMKKRGFGAGKWNGFGGKVQTGETIE...,RHEA:31583;RHEA:31575;RHEA:67604;RHEA:67608;RH...
111,D3ZJ25,RHEA:75347;RHEA:70855;RHEA:66324;RHEA:70891;RH...,"{'-': 0.7777, 'RHEA:34923': 0.9976549744606018...",MAVDPPKADPKGVVAVDPTANCGSGLKSREDQGAKAGGCCSSRDQV...,RHEA:70891;RHEA:70875;RHEA:70863;RHEA:70887;RH...
118,P0DUR6,-,{'-': 0.999997},MPHPMVAPALRRRRRSSFNPHVAAVMTGSILAGAMLSLSFIAVPVL...,RHEA:64268
120,P0DUR5,RHEA:64264;-;RHEA:62680,"{'-': 0.7777, 'RHEA:62680': 0.526458, 'RHEA:64...",MSLPIPPRPADNSTSHSLCLTICGYRRPGMSEFDYRQHMTQVSAPM...,RHEA:64264
123,A0A1L9WLD9,RHEA:64232;RHEA:64512;-;RHEA:12020,"{'-': 0.7777, 'RHEA:12020': 0.199639, 'RHEA:64...",MVTYTPPAEAQDLPTKLKVLYFSNEFPTDDLSTLLRRLHSHSKHSS...,RHEA:64232
124,A0A1L9WLF1,RHEA:64236;-,"{'-': 0.7777, 'RHEA:64236': 0.9974650144577026}",MGEGGYRQINKTLNVCAFDDYLVTQQSRLPKLLDIEQLSPRVLRVL...,RHEA:64236
125,A0A1L9WLH9,RHEA:64292;-,"{'-': 0.7777, 'RHEA:64292': 0.99652099609375}",MTVSSAPYCLAGQVALVTGSGRGIGAAIAVELGRLGASVVVNYANS...,RHEA:64292
131,I0DFJ0,RHEA:19497;RHEA:15689;RHEA:12272;-;RHEA:18533,"{'-': 0.7777, 'RHEA:12272': 0.7777, 'RHEA:1568...",MSENLQLSAEEMRQLGYQAVDLIIDHMNHLKSKPVSETIDSDILRN...,RHEA:30339;RHEA:12272;RHEA:18533


In [64]:
res_df_156, res_json_156 = session_chat(chat_bot=cli5, ix=156, system_prompt=system_prompt)

{
    "protein information": {
        "uniprot id": "Q8KDS8",
        "protein amino acid sequence": "MSVESFERFLSRRVLSMQESQTMKITGLAKKMQAEGKDVVSLSAGEPDFPTPENVCEAGIEAIRKGFTRYTANSGIPELKKAIIRKLQRDNGLEYAEDEIIVSNGGKQALANTFLALCDEGDEVIVPAPYWVSFPEMARLAEATPVIVETSIETGYKMTPEQLAAAITPKTRILVLNSPSNPSGAVYNEAEVRALMQVIEGKEIFVLSDEMYDMICYGGVRPFSPARIPEMKPWVIVSNGTSKSYSMTGWRIGYLAAPKWIINACDKIQSQTTSNANSIAQKAAVAALDGDQSIVEQRRAEFEKRRDFMFRELNTISGIECTLPEGAFYIFPSIKGLLGKTFGGKVMKDSTDVAEYLLTEHYVATVPGDAFGAPENLRLSYAASIEELAEAVNRIRKAFS"
    },
    "reaction information": {
        "candidate reaction 1": {
            "reaction id": "RHEA:21824",
            "reaction equation": "2-oxoglutarate + L-aspartate = L-glutamate + oxaloacetate",
            "reaction equation in ChEBI format": "CHEBI:16810 + CHEBI:29991 = CHEBI:29985 + CHEBI:16452",
            "reaction equation in SMILES format": "[O-]C(=O)CCC(=O)C([O-])=O.[NH3+][C@@H](CC([O-])=O)C([O-])=O>>[NH3+][C@@H](CCC([O-])=O)C([O-])=O.[O-]C(=O)CC(=O)C([O-])=O",
         

In [65]:
res_df_156

,reaction_id,selected,rank,confidence,reason
0,RHEA:21824,yes,1.0,0.98,The UniProt ID Q8KDS8 corresponds to Aspartate...
1,RHEA:22880,no,inf,0.35,While this is a related aminotransferase react...
2,RHEA:12601,no,inf,0.30,This reaction is catalyzed by 2-aminoadipate t...
3,RHEA:20445,no,inf,0.20,This reaction involves L-arogenate and is cata...
4,-,no,inf,0.01,Q8KDS8 is a well-characterized aminotransferas...


In [66]:
res_ecrecer[res_ecrecer.rxn_groundtruth!='-'].iloc[130:155]

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
348,Q9VK50,RHEA:15389;RHEA:13665,"{'RHEA:13665': 0.668856, 'RHEA:15389': 0.97908...",MPRSLGNCQLNYSKERMWEPGYLKAKCAELRLESEFRLYRIRLWKS...,RHEA:15389
350,Q8LPL7,RHEA:16001;RHEA:24408;RHEA:28190;-,"{'-': 0.7777, 'RHEA:16001': 0.9868509769439697...",MEWIQSLPKIELHAHLNGSIRDSTLLELARVLGEKGVIVFADVEHV...,RHEA:16001
353,A0A509ALD0,RHEA:12745;RHEA:52644;RHEA:52648;RHEA:23688;-;...,"{'-': 0.7777, 'RHEA:12745': 0.987500011920929,...",MEIPNEEIKFLKKEDIKNINLNGMNKKERYEIWKKIPKVELHCHLD...,RHEA:25025;RHEA:24408
354,K6UCV4,RHEA:12745;RHEA:52644;RHEA:52648;RHEA:23688;RH...,"{'-': 0.7777, 'RHEA:12745': 0.9861199855804443...",MNILQEPIDFLKKDEIKNIDLSQMSKKERYKIWKRIPKCELHCHLD...,RHEA:25025;RHEA:24408
355,Q8IJA9,RHEA:12745;RHEA:52644;RHEA:52648;RHEA:23688;RH...,"{'-': 0.7777, 'RHEA:12745': 0.9844340085983276...",MNCKNMDTSYEIINYLTKDELDIDLSCMDKKERYKIWKRLPKCELH...,RHEA:25025;RHEA:24408
356,A0A0K1SC59,RHEA:12745;RHEA:52644;RHEA:52648;RHEA:24408;RH...,"{'RHEA:12745': 0.9848179817199707, 'RHEA:23688...",MTILHEEINFLKKDELNINLKCLDKKERYKIWRRIPKCELHCHLDL...,RHEA:24408
357,A0A1Y3DYH2,RHEA:24408;RHEA:28190;RHEA:23688;-,"{'-': 0.7777, 'RHEA:23688': 0.976769, 'RHEA:24...",MNILQEPIDFLKKDELKNIDLSQMDKKERYKIWKRIPKCELHCHLD...,RHEA:25025;RHEA:24408
358,A5KE01,RHEA:12745;RHEA:52644;RHEA:52648;RHEA:23688;RH...,"{'-': 0.7777, 'RHEA:12745': 0.985480010509491,...",MNILQEPIDFLKKEELKNIDLSQMSKKERYKIWKRIPKCELHCHLD...,RHEA:25025;RHEA:24408
359,A0A1S3YCW2,RHEA:17641,{'RHEA:17641': 0.999725},MPALGCCVDAAVSPPPGYSFLWDSSLPAPEIFPSGVPPSTNTAVAT...,RHEA:17641
360,A0A1S3WYR7,RHEA:17641,{'RHEA:17641': 0.999783},MPALGCCVDAAVSPPPGYSFLWDSSLPAPEIFPSGVPLSTNTAATT...,RHEA:17641


In [67]:
session_chat(chat_bot=cli5, ix=348, system_prompt=system_prompt)

{
    "protein information": {
        "uniprot id": "Q9VK50",
        "protein amino acid sequence": "MPRSLGNCQLNYSKERMWEPGYLKAKCAELRLESEFRLYRIRLWKSYLLTFFMLHIFVTSVHCALLLATIERRSIIYFDVALSIGCALVLILVLSVNFCDEFIAKHTWYMYASSIFASLTLVFADLTESIYHTYAHSWILGTFYDTYIIYMIYMFLPIHFISGAVLLALLVSGLYILYFVIFIAQGFAQFASALFSVGGMSVDIVHYLCLNLVGIFYRVMNDTVVRSSFLDRHQYIKEKIWLRNARLQEKQLLDSILPPQISLPLQKDIQGRIVMAKQGIHSWTAMERTMAIQIHPDVSILYADVVNYTHLTTTLTVEMLVKVLHDLYGRFDLAAYRYKVQRIKFLGDCYYCVAGLSDPDPDHANNCVILGLSMINHIMEVRDIHGLDINMRIGVHSGNLFAGVIGEAKLQFDIWGLDVTIANVLESTGVPGCVHISGATLNNLDVNRFDIEDGPEEARDHPLLKKYRIRSYIIRQDLHMDDEDSDEFLGDLHSISLCNMGAQPRISDSANQSLRALFHEELREEFRKMPVSAFSPKRLLGICRFNTGKEVPAHQNLNICLTFTDPILERAYLKQTDYMYKYSIILSASVGCSLVYIELMDTQMICSSCFVLPASVATIQCILALIAWYKKYCWTRYGRNNVPHHYNGFSCFIFRIHDKILNSLPIRICIYLFLMISSFFVMCLIVMSCQREEFEMAYIEERLFHYEQEAHICFHPWVTTNMLSLMICLTFTFAHIPIMVKTAVAILETLAYLLLIFFQFDFVFHHSVTTNPYFKSEYAHALLICITFLIMFVKERQIEFTNKVNFNWRVDLRKEENAASLTNHSIIIILNNILPSHIVDVYLNSLAKHELYFENYRMVSVMFAMLINFEMDLRSLRVLNEIIAEFDTLLLFYKEYY

(  reaction_id selected  rank  confidence  \
 0  RHEA:15389      yes   1.0        0.95   
 1  RHEA:13665       no   inf        0.20   
 
                                               reason  
 0  The UniProt ID Q9VK50 corresponds to the Droso...  
 1  The protein lacks the conserved motifs and act...  ,
 {'results': [{'reaction_id': 'RHEA:15389',
    'selected': 'yes',
    'rank': 1,
    'confidence': 0.95,
    'reason': "The UniProt ID Q9VK50 corresponds to the Drosophila melanogaster protein 'Adenylate cyclase type 3' (AC3), which catalyzes the conversion of ATP to 3',5'-cyclic AMP and diphosphate (EC 4.6.1.1). The protein sequence contains conserved domains characteristic of class III adenylyl cyclases, including the CYTH domain and key residues involved in ATP binding and catalysis. There is strong literature and database support for this function."},
   {'reaction_id': 'RHEA:13665',
    'selected': 'no',
    'confidence': 0.2,
    'reason': 'The protein lacks the conserved motifs

In [13]:
session_chat(chat_bot=cli5, ix=89, system_prompt=system_prompt)

NameError: name 'rxn' is not defined

## 不带UniProt ID 的例子

In [85]:
system_prompt_template_no_uid ="""
You are a biochemical expert. Given a protein sequence (with or without a UniProt ID) and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein. You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.

Key considerations:
	1. **ID-Based Annotation:**
        •	If a UniProt ID is provided, first retrieve functional annotations and known EC numbers from UniProt.
	2. **allback Sequence Analysis:**
        •	If no UniProt ID is available, automatically perform:
            a. A BLAST (or DIAMOND) search against UniProt/NCBI NR, extracting the top 5 homologs and their EC/functional annotations.
            b. A Pfam/InterProScan domain scan to identify characteristic catalytic domains (e.g., PLP-dependent aminotransferase family).
            c. A motif scan for key active-site residues (e.g., PLP-binding lysine motifs, signature sequence patterns).
	3.**Non-Catalytic Cases:**
        •	A reaction represented by a single dash (“-”) indicates a predicted lack of catalytic activity.
        •	If all valid reactions are implausible after ID-based or fallback analysis, select the dash (“-”) reaction.
	4.**Multiple Reactions:**
        •	Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.
	5.**Ranking:**
        •	Assign a likelihood-based ranking to selected reactions (1 = most likely).
	6.**Justification:**
        •	Provide a brief explanation for each decision, referencing database annotations, conserved domains, active-site residues, structural motifs, or known catalytic mechanisms.
	7.**Confidence Score:**
        •	Assign a confidence score between 0 and 1 based on how well the sequence and analysis support each reaction.
        •	If no valid reaction is likely, assign the highest confidence to the dash (“-”) reaction.
Output format (strictly JSON):
[
  {
    "reaction_id": "xxx",
    "selected": "yes" or "no",
    "rank": <integer>,     // Only if selected = "yes"
    "confidence": <float>, // Between 0 and 1
    "reason": "Explanation based on sequence and reaction data."
  },
  ...
]

⚠️ Only output the key “results” in the JSON array. Do not include any extra explanation or metadata.

INPUT:
"""

In [83]:
print(system_prompt)


You are a biochemical expert. Given a protein sequence and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein.
You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.
Key considerations:
1. **Non-Catalytic Cases:**
   - A reaction represented by a single dash ("-") indicates a predicted lack of catalytic activity.
   - If all reactions are marked as "-", or none of the valid reactions are plausible, select the dash ("-") reaction to indicate non-catalytic function.

2. **Multiple Reactions:**
   - Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.

3. **Ranking:**
   - Assign a likelihood-based ranking to selected reactions (1 = most likely).

4. **Justification:**
   - Provide a brief explanation for each decision, referencing conserved domains, active site residues, structural mo

## ---对的例子找解释

In [72]:
res_ecrecer[(res_ecrecer.RXNRECer==res_ecrecer.rxn_groundtruth) & (res_ecrecer.rxn_groundtruth == '-')]

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
0,A9JLI2,-,{'-': 0.999997},MLGLQIFTLLSIPTLLYTYEIEPLERTSTPPEKEFGYWCTYANHCR...,-
1,A9JLI3,-,{'-': 0.999998},MRFFSYLGLLLAGLTSLQGFSTDNLLEEELRYWCQYVKNCRFCWTC...,-
2,A9JLI5,-,{'-': 1.0},MLVIFLGILGLLANQVLGLPTQAEGHLRSTDNPPQEELGYWCTYME...,-
3,A9JLI7,-,{'-': 0.999997},MLVIILGVIGLLANQVLGLPTQAGGHLRSTDNPPQEELGYWCTYME...,-
4,B5KVH4,-,{'-': 0.999998},MAKPILLSIYLCLIIVALFNGCLAQSGGRQQHKFGQCQLNRLDALE...,-
...,...,...,...,...,...
13646,P0DW91,-,{'-': 0.999996},MSGAEEAGGGGPAAGPAGSVPAGVGVGAGAGAGVGVGAGPGAAAGP...,-
13647,P0DTL6,-,{'-': 0.999997},MSGAEEAGGGGPAAGPAGSVPAGVGVGVGAGPGAAAGQAAAAALGE...,-
13648,P0DW87,-,{'-': 0.999997},MSGAEEAGGGGPAAGPAGAVPAGVGVGAGPGAAAGPAAAALGEAAG...,-
13649,P0DW89,-,{'-': 0.999997},MSGAEEAGGGGPAAGPAGAVPAGVGVGVGPGAAAGPAAAALGEAAG...,-


In [91]:
system_prompt_template_get_evidence = """
You are a biochemical expert. Given a protein sequence and its predicted catalyzed-reaction results (in JSON format)—where \"-\" denotes a non-catalytic protein and a list denotes predicted reactions—provide concise evidence and interpretation for each prediction.

You may consult UniProt, BRENDA, KEGG, ExPASy, and other authoritative sources.

Key considerations:
0. Sequence-Level Annotation:
   - (Performed upstream; do not reiterate these steps in your output.)

1. Non-Catalytic Predictions:
   - \"-\" indicates no catalytic activity. Provide direct evidence (e.g., absence of catalytic motifs).

2. Multiple Reactions:
   - If multiple reactions are predicted, furnish evidence for each.

3. Ranking:
   - Order predicted reactions by likelihood (1 = most likely).

4. Justification:
   - Cite conserved domains, active-site residues, structural motifs, homology, or known mechanisms.
   - Do NOT describe the analysis pipeline (BLAST, Pfam, motif scans). Assume it has already been done upstream.

5. Confidence Score:
   - Assign a score [0.0-1.0] reflecting how strongly sequence features support each reaction.
   - If \"-\" is selected, give it the highest confidence.

6. Literature References:
   - If you invoke published studies or databases as evidence, cite real, verifiable references (authors, journal, year, DOI). Do not fabricate citations.

Output (strict JSON using only the key \"results\"):
[
  {
    "reaction_id": "RHEA:XXXXX",
    "confidence": 0.00-1.00,
    "reason": "Concise evidence summary (no pipeline details)."
  },
  …
]
"""

In [155]:
rxn_bank =  pd.read_feather(cfg.FILE_RHEA_REACTION)

def session_chat_getEv(chat_bot, ix, system_prompt, rxn_bank=rxn_bank,uniprot_id='-', debug=True):
    # Extract inputs
    if uniprot_id !='-':
        uniprot_id = res_ecrecer.iloc[ix].uniprot_id
    else:
        uniprot_id = '-'
    seq = res_ecrecer.iloc[ix].seq
    rxn_id = res_ecrecer.iloc[ix].RXNRECer
    rxns = rxn_id.split(cfg.SPLITER)

    # Build the Python object
    query_info = {
        "protein information": {
            "uniprot id": uniprot_id,
            "protein amino acid sequence": seq
        },
        "reaction information": {
            f"predicted reaction {i+1}": get_rxn_detail(rxn, rxn_bank)
            for i, rxn in enumerate(rxns)
        }
    }

    # Serialize to a JSON string
    input_str = "INPUT:\n" + json.dumps(query_info, indent=2)

    if debug:
        print("=== SYSTEM PROMPT ===")
        print(system_prompt)
        print("=== USER MESSAGE ===")
        print(input_str)

    # Pass a string into the chat API
    response = chat_bot.chat(
        message=input_str,
        system_prompt=system_prompt,
        response_format={"type": "json_object"}
    )

    # The `.content` should now be a JSON string
    content_str = response.choices[0].message.content
    if debug:
        print("=== RAW RESPONSE CONTENT ===")
        print(content_str)

    # Parse the JSON
    records = json.loads(content_str)
    df_results = pd.DataFrame(records["results"]).fillna(float("inf"))

    return df_results, records

In [150]:
df_results, records = session_chat_getEv(chat_bot=cli5, ix=4, system_prompt=system_prompt_template_get_evidence)

=== SYSTEM PROMPT ===

You are a biochemical expert. Given a protein sequence and its predicted catalyzed-reaction results (in JSON format)—where "-" denotes a non-catalytic protein and a list denotes predicted reactions—provide concise evidence and interpretation for each prediction.

You may consult UniProt, BRENDA, KEGG, ExPASy, and other authoritative sources.

Key considerations:
0. Sequence-Level Annotation:
   - (Performed upstream; do not reiterate these steps in your output.)

1. Non-Catalytic Predictions:
   - "-" indicates no catalytic activity. Provide direct evidence (e.g., absence of catalytic motifs).

2. Multiple Reactions:
   - If multiple reactions are predicted, furnish evidence for each.

3. Ranking:
   - Order predicted reactions by likelihood (1 = most likely).

4. Justification:
   - Cite conserved domains, active-site residues, structural motifs, homology, or known mechanisms.
   - Do NOT describe the analysis pipeline (BLAST, Pfam, motif scans). Assume it has a

In [140]:
df_results

,reaction_id,confidence,reason
0,-,1.0,The UniProt entry B5KVH4 corresponds to a prot...


In [151]:
records

{'results': [{'reaction_id': '-',
   'confidence': 1.0,
   'reason': 'The protein sequence lacks conserved catalytic motifs, active-site residues, or domains characteristic of known enzymes. No evidence of catalytic function is present based on sequence features (e.g., absence of serine protease, kinase, or oxidoreductase signatures).'}]}

In [152]:
res_ecrecer[(res_ecrecer.RXNRECer==res_ecrecer.rxn_groundtruth) & (res_ecrecer.rxn_groundtruth != '-')]

,uniprot_id,RXNRECer,RXNRECer_with_prob,seq,rxn_groundtruth
162,B6HLT9,RHEA:20720,{'RHEA:20720': 1.0},MLHILCQGTPFEIGYEHGSAAKAVIARSIDFAVDLIRGKTKKTDEE...,RHEA:20720
173,Q82DR2,RHEA:21824,{'RHEA:21824': 0.998798},MSAATPPTERRVSARVGAISESATLAVDAKAKALKAAGRPVIGFGA...,RHEA:21824
252,P96890,RHEA:13501,{'RHEA:13501': 0.998851},MASHAGSRIARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPD...,RHEA:13501
264,A0A8B0RBM2,RHEA:21164,{'RHEA:21164': 0.996007},MELSVIALLLLGFVNFSWQNEQAPRVKTSLGDIRGYYKISRHGRKY...,RHEA:21164
359,A0A1S3YCW2,RHEA:17641,{'RHEA:17641': 0.999725},MPALGCCVDAAVSPPPGYSFLWDSSLPAPEIFPSGVPPSTNTAVAT...,RHEA:17641
...,...,...,...,...,...
13126,A0A1D8PDV7,RHEA:12709,{'RHEA:12709': 0.999987006187439},MATLSQPQSALPKTKIATTFGLSKDLKSPISVKVCYLECTRNNVSL...,RHEA:12709
13127,Q6FSR7,RHEA:12709,{'RHEA:12709': 0.9983569979667664},MSGKSVSFYVSENIDVPLQIKILSLKGKKRQLRASEKLIDPQLSLT...,RHEA:12709
13312,Q949W8,RHEA:10964,{'RHEA:10964': 0.995343029499054},MADLSLPPDSLFLGFDSSTQSMKATVLDSNLNIIKTELVHFDSDLP...,RHEA:10964
13336,A3SR25,RHEA:24204,{'RHEA:24204': 0.996512},MLFRASQPEDKPMKMTTEEAFVKTLQMHGIQHAFGIIGSAMMPISD...,RHEA:24204


In [153]:
df_results, records = session_chat_getEv(chat_bot=cli5, ix=162, system_prompt=system_prompt_template_get_evidence)

=== SYSTEM PROMPT ===

You are a biochemical expert. Given a protein sequence and its predicted catalyzed-reaction results (in JSON format)—where "-" denotes a non-catalytic protein and a list denotes predicted reactions—provide concise evidence and interpretation for each prediction.

You may consult UniProt, BRENDA, KEGG, ExPASy, and other authoritative sources.

Key considerations:
0. Sequence-Level Annotation:
   - (Performed upstream; do not reiterate these steps in your output.)

1. Non-Catalytic Predictions:
   - "-" indicates no catalytic activity. Provide direct evidence (e.g., absence of catalytic motifs).

2. Multiple Reactions:
   - If multiple reactions are predicted, furnish evidence for each.

3. Ranking:
   - Order predicted reactions by likelihood (1 = most likely).

4. Justification:
   - Cite conserved domains, active-site residues, structural motifs, homology, or known mechanisms.
   - Do NOT describe the analysis pipeline (BLAST, Pfam, motif scans). Assume it has a

In [154]:
records

{'results': [{'reaction_id': 'RHEA:20720',
   'confidence': 0.95,
   'reason': 'The protein sequence contains conserved motifs and active-site residues characteristic of acyltransferases involved in penicillin biosynthesis, specifically isopenicillin N acyltransferase (EC 2.3.1.164). Key residues (e.g., Ser, His, Asp triad) and the overall domain architecture match those found in characterized isopenicillin N acyltransferases (UniProt: P00811, PDB: 1QME). This supports the predicted reaction: conversion of isopenicillin N and phenylacetyl-CoA to penicillin G. See: Coque et al., J. Biol. Chem. 267: 6262-6270 (1992), PMID: 1556132; UniProt P00811.'}]}

# 大肠Trembl 例子

In [80]:
case_ecoli_res = pd.read_excel('/hpcfs/fhome/shizhenkun/codebase/RXNRECer/case/ecoli/res/rxnrecer_pred_ecoli_20250520.xlsx')
data_row = case_ecoli_res[case_ecoli_res.uniprot_id =='A0A1V1IGJ0']
data_row

,uniprot_id,protein_name,reviewed,ec_in_uniprot,rxn_id_in_uniprot,rxn_equation_in_uniprot,gene_id_bigg,RXNRECer,RXNRECer_with_prob,ECRECer_equations,...,ECRECER_rhea2bigg_equations,gene_in_iml1515,gene_bigg_rxn,pred_uniprot_correct,ECRECER_pred_EC,RXN4ECRECER_pred_EC,RXNRECer_prob,RXNRECerTrans2EC,RXN_NUM_RXNRECER,RXN_NUM_Uniprot
2990,A0A1V1IGJ0,Guanosine-inosine kinase (EC 2.7.1.73),unreviewed,2.7.1.73,RHEA:27710;RHEA:21140,"['ATP + guanosine = ADP + GMP + H(+)', 'ATP + ...",-,RHEA:27710;RHEA:21140,"{'RHEA:27710': 0.999939, 'RHEA:21140': 0.999939}","['ATP + guanosine = ADP + GMP + H(+)', 'ATP + ...",...,"[['ins_m + atp_m <-> h_m + imp_m + adp_m'], ['...",False,-,True,2.7.1.73,"['ATP + guanosine = ADP + GMP + H(+)', 'ATP + ...",0.999939,"['EC:2.7.1.73', 'EC:2.7.1.73']",2,2


In [11]:
query_str_user = make_make_user_query(uniprot_id=data_row.uniprot_id.values[0],
                                      seq=uptool.get_uniprot_records_by_ids([data_row.uniprot_id.values[0]]).seq[0], 
                                      rxn_id=data_row.RXNRECer.values[0], 
                                      rxn_bank=rxn,
                                      )
print(system_prompt,query_str_user)

NameError: name 'make_make_user_query' is not defined

In [10]:
APIKEY = os.environ.get("OPENROUTER_API_KEY")
chat_bot = llmchat.Chat(name='openai/gpt-4.1', url='https://openrouter.ai/api/v1', api_key = APIKEY)
response = chat_bot.chat(message=query_str_user, system_prompt=system_prompt_template_get_evidence, response_format={"type": "json_object"})   # 关键行，指定返回格式为json_object)

NameError: name 'query_str_user' is not defined

In [ ]:
def session_chat(chat_bot, ix, system_prompt, debug=True):
    
   
    response = chat_bot.chat(message=query_user, system_prompt=system_prompt, response_format={"type": "json_object"})   # 关键行，指定返回格式为json_object)
    print(response)
    records = json.loads(response.choices[0].message.content)
    
    json_results = records
    df_results = pd.DataFrame(records['results']).fillna(float('inf'))
    
    
    if debug:

         print(response)
         print(json_results)
    
    return df_results, json_results



In [93]:
print(response)

ChatCompletion(id='gen-1747902843-iofO89c4kJyINzQonqh6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "results": [\n    {\n      "reaction_id": "RHEA:27710",\n      "confidence": 0.95,\n      "reason": "The protein sequence contains conserved motifs characteristic of nucleoside kinases, including the GXGXXG ATP-binding motif and residues associated with guanosine kinase activity. Homology to characterized guanosine kinases (EC 2.7.1.73) is high (see UniProt Q9H0U4, PDB: 2Q0U), and the substrate specificity loop matches those known to prefer guanosine. Literature supports this function in similar bacterial proteins (Bzowska et al., Acta Biochim Pol, 2000, PMID: 11079780)."\n    },\n    {\n      "reaction_id": "RHEA:21140",\n      "confidence": 0.80,\n      "reason": "The sequence also shares homology with nucleoside kinases capable of phosphorylating inosine, but key substrate recognition residues more closely match guanosine-

In [9]:
p_sys_fine-grained_reranking_with_uniprotid = """  
You are a biochemical expert. Given a protein sequence and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein.
You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.
Key considerations:
1. **Non-Catalytic Cases:**
   - A reaction represented by a single dash ("-") indicates a predicted lack of catalytic activity.
   - If all reactions are marked as "-", or none of the valid reactions are plausible, select the dash ("-") reaction to indicate non-catalytic function.

2. **Multiple Reactions:**
   - Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.

3. **Ranking:**
   - Assign a likelihood-based ranking to selected reactions (1 = most likely).

4. **Justification:**
   - Provide a brief explanation for each decision, referencing conserved domains, active site residues, structural motifs, or known catalytic mechanisms.

5. **Confidence Score:**
   - Assign a confidence score between 0 and 1 based on how well the protein sequence aligns with known catalytic features for each reaction.
   - If no valid reaction is likely, assign the highest confidence to the dash ("-") reaction.

Output format (strictly JSON):
[
    {
        "reaction_id": "xxx",
        "selected": "yes" or "no",
        "rank": <integer>,     // Only for selected = "yes"
        "confidence": <float>, // Between 0 and 1
        "reason": "Explanation based on sequence and reaction data."
    },
    ...
]

⚠️ Only use the key 'results' in the output. Do not include any extra explanation or metadata.

INPUT:
"""

p_sys_fine_grained_reranking_without_uniprotid = """ 
You are a biochemical expert. Given a protein sequence (with or without a UniProt ID) and a set of candidate catalyzed reactions (in JSON format), analyze and determine which reaction(s) are most likely catalyzed by the protein. You may consult relevant biochemical databases including UniProt, BRENDA, and ExPASy to retrieve supporting information.

Key considerations:
	1. **ID-Based Annotation:**
        •	If a UniProt ID is provided, first retrieve functional annotations and known EC numbers from UniProt.
	2. **allback Sequence Analysis:**
        •	If no UniProt ID is available, automatically perform:
            a. A BLAST (or DIAMOND) search against UniProt/NCBI NR, extracting the top 5 homologs and their EC/functional annotations.
            b. A Pfam/InterProScan domain scan to identify characteristic catalytic domains (e.g., PLP-dependent aminotransferase family).
            c. A motif scan for key active-site residues (e.g., PLP-binding lysine motifs, signature sequence patterns).
	3.**Non-Catalytic Cases:**
        •	A reaction represented by a single dash (“-”) indicates a predicted lack of catalytic activity.
        •	If all valid reactions are implausible after ID-based or fallback analysis, select the dash (“-”) reaction.
	4.**Multiple Reactions:**
        •	Proteins may have multiple active sites. Select more than one reaction if supported by sequence features.
	5.**Ranking:**
        •	Assign a likelihood-based ranking to selected reactions (1 = most likely).
	6.**Justification:**
        •	Provide a brief explanation for each decision, referencing database annotations, conserved domains, active-site residues, structural motifs, or known catalytic mechanisms.
	7.**Confidence Score:**
        •	Assign a confidence score between 0 and 1 based on how well the sequence and analysis support each reaction.
        •	If no valid reaction is likely, assign the highest confidence to the dash (“-”) reaction.
Output format (strictly JSON):
[
  {
    "reaction_id": "xxx",
    "selected": "yes" or "no",
    "rank": <integer>,     // Only if selected = "yes"
    "confidence": <float>, // Between 0 and 1
    "reason": "Explanation based on sequence and reaction data."
  },
  ...
]

⚠️ Only output the key “results” in the JSON array. Do not include any extra explanation or metadata.

INPUT:
"""





SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3790980079.py, line 1)

In [13]:
import os
# os.path.abspath('__file__')
# # os.getcwd()
os.path.dirname(os.path.realpath('__file__'))

'/hpcfs/fhome/shizhenkun/codebase/RXNRECer/case/llm'

In [15]:
os.path.abspath('__file__')

'/hpcfs/fhome/shizhenkun/codebase/RXNRECer/case/llm/__file__'

In [16]:
os.path.abspath('aaa')

'/hpcfs/fhome/shizhenkun/codebase/RXNRECer/case/llm/aaa'